In [3]:
"""

This program shows how to retrieve info for a single source from the frost.met.no service.

The HTTP request essentially consists of the following components:
  - the endpoint, frost.met.no/sources
  - the source ID to get information for
  - the client ID used for authentication

The source ID is read from a command-line argument, while the client ID is read from
the environment variable CLIENTID.

Save the program to a file example.py, make it executable (chmod 755 example.py),
and run it e.g. like this:

  $ CLIENTID=8e6378f7-b3-ae4fe-683f-0db1eb31b24ec ./example.py SN18700

or like this to get info for sources matching a pattern:

  $ CLIENTID=8e6378f7-b3-ae4fe-683f-0db1eb31b24ec ./example.py SN187*

(Note: the client ID used in the example should be replaced with a real one)

The program has been tested on the following platforms:
  - Python 2.7.3 on Ubuntu 12.04 Precise
  - Python 2.7.12 and 3.5.2 on Ubuntu 16.04 Xenial

"""

import sys, os
import requests # See http://docs.python-requests.org/


client_id = 'TOBEFILLED'

source_id='SN25830'

# issue an HTTP GET request
r = requests.get(
        'https://frost.met.no/sources/v0.jsonld',
        {'ids': source_id},
        auth=(client_id, '')
    )

def codec_utf8(s):
    #return s.encode('utf-8').decode('utf-8') # should be used for Python 3
    return s.encode('utf-8') # should be used for Python 2

# extract some data from the response
if r.status_code == 200:
        for item in r.json()['data']:
            sys.stdout.write('ID: {}\n'.format(item['id']))
            sys.stdout.write('Name: {}\n'.format(codec_utf8(item['name'])))
            if 'geometry' in item:
                sys.stdout.write('longitude: {}\n'.format(item['geometry']['coordinates'][0]))
                sys.stdout.write('latitude: {}\n'.format(item['geometry']['coordinates'][1]))
            if 'municipality' in item:
                sys.stdout.write('Municipality: {}\n'.format(codec_utf8(item['municipality'])))
            if 'county' in item:
                sys.stdout.write('County: {}\n'.format(codec_utf8(item['county'])))
            sys.stdout.write('Country: {}\n'.format(codec_utf8(item['country'])))
            if 'externalIds' in item:
                for ext_id in item['externalIds']:
                    sys.stdout.write('external ID: {}\n'.format(ext_id))
            else:
                sys.stdout.write('no external IDs found\n')
else:
        sys.stdout.write('error:\n')
        sys.stdout.write('\tstatus code: {}\n'.format(r.status_code))
        if 'error' in r.json():
            assert(r.json()['error']['code'] == r.status_code)
            sys.stdout.write('\tmessage: {}\n'.format(r.json()['error']['message']))
            sys.stdout.write('\treason: {}\n'.format(r.json()['error']['reason']))
        else:
            sys.stdout.write('\tother error\n')
    


ID: SN25830
Name: b'FINSEVATN'
longitude: 7.527
latitude: 60.5938
Municipality: b'ULVIK'
County: b'HORDALAND'
Country: b'Norge'
external ID: 01350
external ID: 10.249.0.208


In [19]:
"""

This program shows how to retrieve a time series of observations from the following
combination of source, element and time range:

source:     SN18700
element:    mean(wind_speed P1D)
time range: 2010-04-01 .. 2010-05-31

The time series is written to standard output as lines of the form:

  <observation time as date/time in ISO 8601 format> \
  <observation time as seconds since 1970-01-01T00:00:00> \
  <observed value>

Save the program to a file example.py, make it executable (chmod 755 example.py),
and run it e.g. like this:

  $ CLIENTID=8e6378f7-b3-ae4fe-683f-0db1eb31b24ec ./example.py

(Note: the client ID used in the example should be replaced with a real one)

The program has been tested on the following platforms:
  - Python 2.7.3 on Ubuntu 12.04 Precise
  - Python 2.7.12 and 3.5.2 on Ubuntu 16.04 Xenial

"""

import sys, os
import dateutil.parser as dp
import requests # See http://docs.python-requests.org/

# issue an HTTP GET request
r = requests.get(
        'https://frost.met.no/observations/v0.jsonld',
#        {'sources': 'SN18700', 'elements': 'mean(wind_speed P1D)', 'referencetime': '2010-04-01/2010-06-01'},
        {'sources': 'SN25830', 'elements': 'surface_temperature', 'referencetime': '2018-01-01/2018-01-26'},
        auth=(client_id, '')
    )

# surface_air_pressure
# air_temperature


# extract the time series from the response
if r.status_code == 200:
        for item in r.json()['data']:
            iso8601 = item['referenceTime']
            #print(iso8601)
            #secsSince1970 = dp.parse(iso8601).strftime('%s')
            #sys.stdout.write('{} {} {}\n'.format(iso8601, secsSince1970, item['observations'][0]['value']))
            print(iso8601, item['observations'][0]['value'])
else:
        sys.stdout.write('error:\n')
        sys.stdout.write('\tstatus code: {}\n'.format(r.status_code))
        if 'error' in r.json():
            assert(r.json()['error']['code'] == r.status_code)
            sys.stdout.write('\tmessage: {}\n'.format(r.json()['error']['message']))
            sys.stdout.write('\treason: {}\n'.format(r.json()['error']['reason']))
        else:
            sys.stdout.write('\tother error\n')
    

2018-01-01T00:00:00.000Z -8.1
2018-01-01T01:00:00.000Z -7.8
2018-01-01T02:00:00.000Z -7.7
2018-01-01T03:00:00.000Z -6.8
2018-01-01T04:00:00.000Z -4.8
2018-01-01T05:00:00.000Z -4.3
2018-01-01T06:00:00.000Z -4
2018-01-01T07:00:00.000Z -3.7
2018-01-01T08:00:00.000Z -3.8
2018-01-01T09:00:00.000Z -3.7
2018-01-01T10:00:00.000Z -3.7
2018-01-01T11:00:00.000Z -3.4
2018-01-01T12:00:00.000Z -3.4
2018-01-01T13:00:00.000Z -3.5
2018-01-01T14:00:00.000Z -3.5
2018-01-01T15:00:00.000Z -6.9
2018-01-01T16:00:00.000Z -4.7
2018-01-01T17:00:00.000Z -4
2018-01-01T18:00:00.000Z -4.3
2018-01-01T19:00:00.000Z -3.8
2018-01-01T20:00:00.000Z -4.8
2018-01-01T21:00:00.000Z -5.1
2018-01-01T22:00:00.000Z -5.6
2018-01-01T23:00:00.000Z -5.5
2018-01-02T00:00:00.000Z -7
2018-01-02T01:00:00.000Z -5.6
2018-01-02T02:00:00.000Z -4.9
2018-01-02T03:00:00.000Z -5
2018-01-02T04:00:00.000Z -5.2
2018-01-02T05:00:00.000Z -4.8
2018-01-02T06:00:00.000Z -5.1
2018-01-02T07:00:00.000Z -5.3
2018-01-02T08:00:00.000Z -5.7
2018-01-02T09:00:0